<a href="https://colab.research.google.com/github/Aryan-401/GoogleColabProjects/blob/main/Sentiment_Analysis_%F0%9F%A4%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
import torch
import torch.nn.functional as F

In [3]:
classifier_v0 = pipeline("sentiment-analysis")
test_many = ["This is pretty good", "I dont hate it", "There is too much salt"]
test_one = "I can't believe that I got the part"
result = classifier_v0(test_many)
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9998430013656616}, {'label': 'NEGATIVE', 'score': 0.5758727192878723}, {'label': 'NEGATIVE', 'score': 0.9955743551254272}]


In [4]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
classifier_v1 = pipeline("sentiment-analysis", model = model_name)
result = classifier_v1(test_many)
print(result)

[{'label': 'POSITIVE', 'score': 0.9998430013656616}, {'label': 'NEGATIVE', 'score': 0.5758727192878723}, {'label': 'NEGATIVE', 'score': 0.9955743551254272}]


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier_v2 = pipeline("sentiment-analysis", model =  model, tokenizer=tokenizer)
result = classifier_v2(test_many)
print(result)

[{'label': 'POSITIVE', 'score': 0.9998430013656616}, {'label': 'NEGATIVE', 'score': 0.5758727192878723}, {'label': 'NEGATIVE', 'score': 0.9955743551254272}]


In [6]:
tokens = tokenizer.tokenize(test_one)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer(test_one)


print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")
print(f"Input IDs: {input_ids}")

Tokens: ['i', 'can', "'", 't', 'believe', 'that', 'i', 'got', 'the', 'part']
Token IDs: [1045, 2064, 1005, 1056, 2903, 2008, 1045, 2288, 1996, 2112]
Input IDs: {'input_ids': [101, 1045, 2064, 1005, 1056, 2903, 2008, 1045, 2288, 1996, 2112, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
batch = tokenizer(test_many, padding=True, truncation=True, max_length=512, return_tensors='pt')
print(f"Input IDs: {batch}")

with torch.no_grad():
    outputs = model(**batch)
    print(f"Outputs: {outputs}")
    predictions = F.softmax(outputs.logits, dim=1)
    print(f"Predictions: {predictions}")
    labels = torch.argmax(predictions, dim = 1)
    print(f"Labels 1: {labels}")
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(f"Labels 2: {labels}")

Input IDs: {'input_ids': tensor([[ 101, 2023, 2003, 3492, 2204,  102,    0],
        [ 101, 1045, 2123, 2102, 5223, 2009,  102],
        [ 101, 2045, 2003, 2205, 2172, 5474,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}
Outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-4.1858,  4.5737],
        [ 0.2723, -0.0335],
        [ 2.9216, -2.4943]]), hidden_states=None, attentions=None)
Predictions: tensor([[1.5694e-04, 9.9984e-01],
        [5.7587e-01, 4.2413e-01],
        [9.9557e-01, 4.4256e-03]])
Labels 1: tensor([1, 0, 0])
Labels 2: ['POSITIVE', 'NEGATIVE', 'NEGATIVE']


In [8]:
tokenizer.save_pretrained("saved_hf")
model.save_pretrained("saved_hf")

In [10]:
model_name = 'oliverguhr/german-sentiment-bert'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
test_many_german = ["Das ist ziemlich gut", "Ich hasse es nicht", "Es ist zu viel Salz"]

In [15]:
batch = tokenizer(test_many_german, padding=True, truncation=True, max_length=512, return_tensors='pt')
batch = torch.tensor(batch['input_ids'])
print(f"Input IDs: {batch}")

with torch.no_grad():
    outputs = model(batch)
    print(f"Outputs: {outputs}")
    predictions = F.softmax(outputs.logits, dim=1)
    print(f"Predictions: {predictions}")
    labels = torch.argmax(predictions, dim = 1)
    print(f"Labels 1: {labels}")
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(f"Labels 2: {labels}")

<ipython-input-15-16ee9002fba8>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch['input_ids'])
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Input IDs: tensor([[    3,   295,   127,  9431,  1522,     4,     0],
        [    3,  1671,   312, 11370,   229,   149,     4],
        [    3,   482,   127,    81,   870,  4979,     4]])
Outputs: SequenceClassifierOutput(loss=None, logits=tensor([[ 3.6143,  0.1325, -4.7010],
        [ 3.0149,  0.2193, -3.6925],
        [-0.8959,  2.3623, -1.9595]]), hidden_states=None, attentions=None)
Predictions: tensor([[9.6994e-01, 2.9826e-02, 2.3738e-04],
        [9.4135e-01, 5.7495e-02, 1.1502e-03],
        [3.6567e-02, 9.5081e-01, 1.2623e-02]])
Labels 1: tensor([0, 0, 1])
Labels 2: ['positive', 'positive', 'negative']
